In [27]:
import boto3
import time 
import sys
import os
import json

class S3Bucket(object):
    def __init__(self, bucket_name):
        try:
            with open("config.json", 'r', encoding="utf8") as config:
                json_config = json.load(config)
        except FileNotFoundError as e:
            print('Configuration file not found.')
            sys.exit()
            
        self.aws_secret_access_key = json_config["credentials"]["aws_secret_access_key"]
        self.aws_access_key_id = json_config["credentials"]["aws_access_key_id"]
        self.region_name = json_config["credentials"]["region_name"]
        self.bucket_name = bucket_name
        
    def aws_connection(self):
        """Make connection with AWS S3 Bucket."""
        # create S3 Client
        self.s3_client = boto3.client('s3', aws_access_key_id=self.aws_access_key_id,
                                      aws_secret_access_key=self.aws_secret_access_key,
                                      region_name=self.region_name)
        return self.s3_client
        
    def create_s3_bucket(self):
        """Cretae a private S3 Bucket on AWS with the given Bucket name."""
        response = self.s3_client.create_bucket(ACL='private', 
                            Bucket=self.bucket_name,
                            CreateBucketConfiguration={'LocationConstraint': self.region_name})
        return response 
    
    def list_files(self, path):
        """Get the list of files that has to be uploaded."""
        files = [file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))]
        for file in files: # You could shorten this to one line, but it runs on a bit.
            print(file)
        return files

    def upload_s3_files(self, path, files):
        """ Upload files to s3 Bucket from the given directory."""
        # upload file to S3 Bucket 
        for file in files:
            filepath = os.path.join(path, file)
            response = self.s3_client.upload_file(filepath, self.bucket_name, file)
        print('All files has been successfully uploaded.')
        
    def delete_bucket(self, bucket_name):
        # get the list of files that has been uploaded to S3 bucket 
        response = self.s3_client.list_objects_v2(
            Bucket= bucket_name,
        )

        # delete the s3 bucket files one-by-one 
        while response['KeyCount'] > 0:
            print('Deleting %d objects from bucket %s' % (len(response['Contents']),bucket_name))
            response = self.s3_client.delete_objects(
                Bucket=bucket_name,
                Delete={
                    'Objects':[{'Key':obj['Key']} for obj in response['Contents']]
                }
            )
            response = self.s3_client.list_objects_v2(
                Bucket=bucket_name,
            )
        # Delete the Bucket once it is empty
        print('Now deleting bucket %s' % bucket_name)
        response = self.s3_client.delete_bucket(
            Bucket=bucket_name
        )

#### Create AWS connection with S3 Bucket

In [28]:
# Pass the unique Bucket name. The name of the Bucket must be unique. 
bucket_name = "myuniquebucketdemo0989"  
s3_obj = S3Bucket(bucket_name)

# Create the aws S3 connection. 
s3_connection = s3_obj.aws_connection()

#### Create S3 Bucket of a given name on AWS S3 Storage 

In [29]:
s3_bucket_response = s3_obj.create_s3_bucket()

#### Get the list of the files on a given path that has to upload to S3 Bucket

In [30]:
# Provide the path of the directory that contained files
path = "C:\\Users\\javed.ahmad\\Documents\\Github Respo\\S3 Bucket"

# Get list of file at given directory path
files = s3_obj.list_files(path)

AWS_Demo_Page.jpg
config.json
download.png
download.txt
Helper.ipynb
main.ipynb


#### Upload files to S3 Bucket that has created.

In [31]:
s3_obj.upload_s3_files(path, files)

All files has been successfully uploaded.


#### Delete Files and S3 Bucket  

In [32]:
s3_obj.delete_bucket(bucket_name)

Deleting 6 objects from bucket myuniquebucketdemo0989
Now deleting bucket myuniquebucketdemo0989
